In [39]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import scikitplot as skplt
import matplotlib.pyplot as plt


In [40]:
text = []
label = []
for i in open('/content/sample_data/imdb_labelled.txt'):
  i = i.rstrip('\n').split('\t')
  text.append(i[0].strip(' '))
  label.append(i[1])

text = np.array(text)
label = np.array(label)

print (text.shape)
print (label.shape)

(1000,)
(1000,)


In [41]:
df_text = pd.DataFrame({'text': text , 'label':label })

In [42]:
df_text.head(5)

,text,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [43]:
# Now we will do preprocessing on the text data using the following
vectorize_layer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens = None,
    standardize = 'lower_and_strip_punctuation',
    split = 'whitespace',
    ngrams = None,
    output_mode = 'int',
    output_sequence_length = None
)

In [44]:
vectorize_layer.adapt(text)

In [45]:
print (vectorize_layer.get_vocabulary())

['', '[UNK]', 'the', 'a', 'and', 'of', 'is', 'this', 'i', 'it', 'to', 'in', 'was', 'movie', 'that', 'film', 'for', 'as', 'but', 'with', 'its', 'one', 'you', 'on', 'are', 'not', 'bad', 'very', 'just', 'so', 'all', 'good', 'at', 'an', 'there', 'be', 'about', 'by', 'have', 'like', 'from', 'if', 'time', 'really', 'his', 'acting', 'out', 'or', 'great', 'even', 'who', 'were', 'he', 'has', 'see', 'characters', 'my', 'most', 'how', 'well', 'no', 'more', 'only', 'movies', 'ever', 'story', 'when', 'some', 'plot', 'made', 'films', 'they', 'dont', 'your', 'best', 'because', 'also', 'than', 'script', 'can', 'way', 'seen', 'other', 'would', 'which', 'what', 'watching', 'me', 'make', 'love', 'her', 'didnt', 'character', 'will', 'think', 'real', 'could', 'any', 'much', 'had', 'every', 'do', 'up', 'too', 'scenes', 'look', 'work', 'wonderful', 'funny', 'cast', 'better', 'actors', 'watch', 'them', 'over', 'never', 'little', 'into', '10', 'show', 'everything', 'totally', 'their', 'here', 'excellent', 'bot

In [46]:
a =  vectorize_layer.get_vocabulary()
print (a)
print (a.index('movie'))

['', '[UNK]', 'the', 'a', 'and', 'of', 'is', 'this', 'i', 'it', 'to', 'in', 'was', 'movie', 'that', 'film', 'for', 'as', 'but', 'with', 'its', 'one', 'you', 'on', 'are', 'not', 'bad', 'very', 'just', 'so', 'all', 'good', 'at', 'an', 'there', 'be', 'about', 'by', 'have', 'like', 'from', 'if', 'time', 'really', 'his', 'acting', 'out', 'or', 'great', 'even', 'who', 'were', 'he', 'has', 'see', 'characters', 'my', 'most', 'how', 'well', 'no', 'more', 'only', 'movies', 'ever', 'story', 'when', 'some', 'plot', 'made', 'films', 'they', 'dont', 'your', 'best', 'because', 'also', 'than', 'script', 'can', 'way', 'seen', 'other', 'would', 'which', 'what', 'watching', 'me', 'make', 'love', 'her', 'didnt', 'character', 'will', 'think', 'real', 'could', 'any', 'much', 'had', 'every', 'do', 'up', 'too', 'scenes', 'look', 'work', 'wonderful', 'funny', 'cast', 'better', 'actors', 'watch', 'them', 'over', 'never', 'little', 'into', '10', 'show', 'everything', 'totally', 'their', 'here', 'excellent', 'bot

In [47]:
# now use it to process some text
# get_vocabulary is used to get the lsit of all the words sorted with descending
# number of times a word is repeated and to encode the input text according to the index
input_text = [['good movie'], ['Mochen Yang']]
vectorize_layer(input_text)

<tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[31, 13],
       [ 1,  1]])>

**Pre processing**

In [48]:
df_text = df_text.astype({'label': 'int32'})
df_text.dtypes

text     object
label     int32
dtype: object

In [49]:
X_train, X_test, y_train, y_test = train_test_split(df_text['text'], df_text['label'], test_size=0.20, random_state=42)

print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)


(800,)
(800,)
(200,)
(200,)


**RNN model**

Now, let's actually build a basic RNN model, by stacking together the text processing layer, an embedding layer, and an RNN layer

In [50]:
model_rnn = keras.Sequential()

# Vectorize layer
model_rnn.add(vectorize_layer)

# Embedding layer
model_rnn.add(keras.layers.Embedding(
    input_dim = len(vectorize_layer.get_vocabulary()),
    output_dim = 64,
    mask_zero = True
))

# RNN layer
model_rnn.add(keras.layers.SimpleRNN(128)) # see note below

# Output layer
model_rnn.add(keras.layers.Dense(1, activation = 'sigmoid'))

In [51]:
# configure training / optimization
model_rnn.compile(loss = keras.losses.BinaryCrossentropy(),
                  optimizer='adam',
                  metrics=['accuracy'])

In [52]:
# training with 20% validation and 10 epochs.
model_rnn.fit(x = X_train, y = y_train, validation_split = 0.2,
              epochs=10, batch_size = 32)

Epoch 1/10
20/20 [==============================] - 2s 36ms/step - loss: 0.7011 - accuracy: 0.5094 - val_loss: 0.6908 - val_accuracy: 0.5500
Epoch 2/10
20/20 [==============================] - 0s 23ms/step - loss: 0.5332 - accuracy: 0.8547 - val_loss: 0.7781 - val_accuracy: 0.5500
Epoch 3/10
20/20 [==============================] - 0s 23ms/step - loss: 0.2968 - accuracy: 0.9266 - val_loss: 1.0367 - val_accuracy: 0.5437
Epoch 4/10
20/20 [==============================] - 0s 24ms/step - loss: 0.1665 - accuracy: 0.9688 - val_loss: 1.1430 - val_accuracy: 0.5500
Epoch 5/10
20/20 [==============================] - 0s 22ms/step - loss: 0.1195 - accuracy: 0.9844 - val_loss: 1.0666 - val_accuracy: 0.5938
Epoch 6/10
20/20 [==============================] - 0s 25ms/step - loss: 0.1221 - accuracy: 0.9781 - val_loss: 0.9534 - val_accuracy: 0.6438
Epoch 7/10
20/20 [==============================] - 0s 24ms/step - loss: 0.0900 - accuracy: 0.9844 - val_loss: 1.2063 - val_accuracy: 0.5312
Epoch 8/10
20

In [53]:
model_rnn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, None, 64)          203264    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               24704     
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 228,097
Trainable params: 228,097
Non-trainable params: 0
_________________________________________________________________


In [54]:
# Lets use this RNN to make predictions on some sentences 
rnn_prediction = (model_rnn.predict(X_test)[:,0] >= 0.5).astype(bool)

print(classification_report(y_test, rnn_prediction, labels=[1, 0]))

              precision    recall  f1-score   support

           1       0.69      0.54      0.61       109
           0       0.56      0.70      0.62        91

    accuracy                           0.61       200
   macro avg       0.62      0.62      0.61       200
weighted avg       0.63      0.61      0.61       200



In [55]:
a = (model_rnn.predict(X_test)[::,0])
print (a.shape)
print (y_test.shape)

(200,)
(200,)


In [56]:
y_test = y_test.reset_index(drop = True)

In [57]:
a = pd.Series(a)
print ((a).dtype)
df_output = pd.DataFrame ({'real_label': y_test , 'predict_prob': a})

float32


In [58]:
#LR curves
#skplt.metrics.plot_roc(df_output['real_label'], df_output['predict_prob'])

# skplt.metrics.plot_cumulative_gain(y_test, model_rnn.predict(X_test))

# skplt.metrics.plot_lift_curve(y_test, model_rnn.predict(X_test))

LSTM network

Lets now try a LSTM network 

In [59]:
model_lstm = keras.Sequential()

model_lstm.add(vectorize_layer)

model_lstm.add(keras.layers.Embedding(
    input_dim = len(vectorize_layer.get_vocabulary()),
    output_dim = 64,
    mask_zero = True
))

model_lstm.add(keras.layers.LSTM(128))

model_lstm.add(keras.layers.Dense(1, activation = 'sigmoid'))



In [60]:
# configure training / optimization
model_lstm.compile(loss = keras.losses.BinaryCrossentropy(),
                   optimizer='adam',
                   metrics=['accuracy'])

In [61]:
# training with 20% validation and 10 epochs.
model_lstm.fit(x = X_train, y = y_train, validation_split = 0.2,
               epochs=10, batch_size = 32)

Epoch 1/10
20/20 [==============================] - 5s 115ms/step - loss: 0.6928 - accuracy: 0.5266 - val_loss: 0.6870 - val_accuracy: 0.5688
Epoch 2/10
20/20 [==============================] - 1s 56ms/step - loss: 0.6353 - accuracy: 0.7828 - val_loss: 0.5744 - val_accuracy: 0.7563
Epoch 3/10
20/20 [==============================] - 1s 53ms/step - loss: 0.4086 - accuracy: 0.8703 - val_loss: 0.6457 - val_accuracy: 0.7063
Epoch 4/10
20/20 [==============================] - 1s 52ms/step - loss: 0.2444 - accuracy: 0.9453 - val_loss: 0.5651 - val_accuracy: 0.7812
Epoch 5/10
20/20 [==============================] - 1s 52ms/step - loss: 0.1535 - accuracy: 0.9656 - val_loss: 0.5024 - val_accuracy: 0.7812
Epoch 6/10
20/20 [==============================] - 1s 54ms/step - loss: 0.0845 - accuracy: 0.9891 - val_loss: 0.6880 - val_accuracy: 0.7875
Epoch 7/10
20/20 [==============================] - 1s 52ms/step - loss: 0.0930 - accuracy: 0.9781 - val_loss: 0.5710 - val_accuracy: 0.7875
Epoch 8/10
2

In [62]:
# Lets use this RNN to make predictions on some sentences 
lstm_prediction = (model_lstm.predict(X_test)[:,0] >= 0.5).astype(bool)

print(classification_report(y_test, lstm_prediction, labels=[1, 0]))

              precision    recall  f1-score   support

           1       0.74      0.80      0.77       109
           0       0.73      0.66      0.69        91

    accuracy                           0.73       200
   macro avg       0.73      0.73      0.73       200
weighted avg       0.73      0.73      0.73       200



Lets build GRU 

In [63]:
model_gru = keras.Sequential()

model_gru.add(vectorize_layer)

model_gru.add(keras.layers.Embedding(
    input_dim = len(vectorize_layer.get_vocabulary()),
    output_dim = 64,
    mask_zero = True
))

model_gru.add(keras.layers.GRU(128))

model_gru.add(keras.layers.Dense(1, activation = 'sigmoid'))

In [64]:
# configure training / optimization
model_gru.compile(loss = keras.losses.BinaryCrossentropy(),
                  optimizer='adam',
                  metrics=['accuracy'])

In [65]:
# training with 20% validation and 10 epochs.
model_gru.fit(x = X_train, y = y_train, validation_split = 0.2,
              epochs=10, batch_size = 32)

Epoch 1/10
20/20 [==============================] - 6s 95ms/step - loss: 0.6925 - accuracy: 0.5234 - val_loss: 0.6881 - val_accuracy: 0.5625
Epoch 2/10
20/20 [==============================] - 1s 40ms/step - loss: 0.6455 - accuracy: 0.7344 - val_loss: 0.6175 - val_accuracy: 0.6750
Epoch 3/10
20/20 [==============================] - 1s 40ms/step - loss: 0.4388 - accuracy: 0.8703 - val_loss: 0.5402 - val_accuracy: 0.7563
Epoch 4/10
20/20 [==============================] - 1s 41ms/step - loss: 0.2051 - accuracy: 0.9547 - val_loss: 0.5419 - val_accuracy: 0.7875
Epoch 5/10
20/20 [==============================] - 1s 42ms/step - loss: 0.0872 - accuracy: 0.9828 - val_loss: 0.6817 - val_accuracy: 0.8000
Epoch 6/10
20/20 [==============================] - 1s 42ms/step - loss: 0.0444 - accuracy: 0.9953 - val_loss: 0.6911 - val_accuracy: 0.8000
Epoch 7/10
20/20 [==============================] - 1s 40ms/step - loss: 0.0258 - accuracy: 0.9969 - val_loss: 0.8537 - val_accuracy: 0.8000
Epoch 8/10
20

In [66]:
# Lets use this RNN to make predictions on some sentences 
lstm_prediction = (model_lstm.predict(X_test)[:,0] >= 0.5).astype(bool)

print(classification_report(y_test, lstm_prediction, labels=[1, 0]))

              precision    recall  f1-score   support

           1       0.74      0.80      0.77       109
           0       0.73      0.66      0.69        91

    accuracy                           0.73       200
   macro avg       0.73      0.73      0.73       200
weighted avg       0.73      0.73      0.73       200



Lets also try to make a bidirectional RNN

In [67]:
model_bilstm = keras.Sequential()

model_bilstm.add(vectorize_layer)

model_bilstm.add(keras.layers.Embedding(
    input_dim = len(vectorize_layer.get_vocabulary()),
    output_dim = 64,
    mask_zero = True
))

model_bilstm.add(keras.layers.Bidirectional(keras.layers.LSTM(128)))

model_bilstm.add(keras.layers.Dense(1, activation = 'sigmoid'))

In [68]:
# configure training / optimization
model_bilstm.compile(loss = keras.losses.BinaryCrossentropy(),
                     optimizer='adam',
                     metrics=['accuracy'])

In [69]:
# training with 20% validation and 10 epochs.
model_bilstm.fit(x = X_train, y = y_train, validation_split = 0.2,
                 epochs = 10, batch_size = 32)

Epoch 1/10
20/20 [==============================] - 10s 227ms/step - loss: 0.6912 - accuracy: 0.5203 - val_loss: 0.6805 - val_accuracy: 0.6875
Epoch 2/10
20/20 [==============================] - 2s 84ms/step - loss: 0.5728 - accuracy: 0.8359 - val_loss: 0.7942 - val_accuracy: 0.7250
Epoch 3/10
20/20 [==============================] - 2s 82ms/step - loss: 0.2838 - accuracy: 0.9250 - val_loss: 0.6850 - val_accuracy: 0.7500
Epoch 4/10
20/20 [==============================] - 2s 84ms/step - loss: 0.1595 - accuracy: 0.9625 - val_loss: 0.6736 - val_accuracy: 0.8000
Epoch 5/10
20/20 [==============================] - 2s 84ms/step - loss: 0.1077 - accuracy: 0.9812 - val_loss: 0.7603 - val_accuracy: 0.7688
Epoch 6/10
20/20 [==============================] - 2s 84ms/step - loss: 0.0823 - accuracy: 0.9812 - val_loss: 0.9663 - val_accuracy: 0.7625
Epoch 7/10
20/20 [==============================] - 2s 84ms/step - loss: 0.0945 - accuracy: 0.9828 - val_loss: 1.1641 - val_accuracy: 0.7688
Epoch 8/10


In [70]:
# Lets use this RNN to make predictions on some sentences 
lstm_prediction = (model_lstm.predict(X_test)[:,0] >= 0.5).astype(bool)

print(classification_report(y_test, lstm_prediction, labels=[1, 0]))

              precision    recall  f1-score   support

           1       0.74      0.80      0.77       109
           0       0.73      0.66      0.69        91

    accuracy                           0.73       200
   macro avg       0.73      0.73      0.73       200
weighted avg       0.73      0.73      0.73       200

